In [1]:
#default_exp core

In [2]:
#export
from nbdev.imports import *

# Notebook core
> Core functions for the notebook module

## Config -

In [3]:
#export
def save_config_file(file, config):
    with file.open('w') as f: yaml.dump(config, f, default_flow_style=False)

In [35]:
# export
class Config:
    "Store the basic information for nbdev to work"
    def __init__(self):
        cfg_path = Path.cwd()
        while cfg_path != Path('/') and not (cfg_path/'config.yml').exists(): cfg_path = cfg_path.parent
        self.config_file = cfg_path/'config.yml'
        assert self.config_file.exists(), "Use `Config.create` to create a `Config` object the first time"
        self.d = self.load_config()

    def __getattr__(self,k):
        if k=='d': raise AttributeError
        if k not in self.d: raise AttributeError
        return self.config_file.parent/self[k] if k.endswith('path') else self[k]

    def __getitem__(self,k):   return self.d[k]
    def __setitem__(self,k,v): self.d[k] = str(v)
    def __contains__(self,k):  return k in self.d

    def load_config(self):
        with open(self.config_file, 'r') as f:
            config = yaml.safe_load(f)
            return config

    @classmethod
    def create(cls, lib_name, git_url, path='.', nb_folder='nbs', lib_folder=None, doc_folder='docs', tst_flags=None):
        path = Path(path)
        if lib_folder is None: lib_folder = lib_name
        if tst_flags is None: tst_flags = []
        config = {'lib_name':  lib_name,
                  'git_url': git_url,
                  'lib_path': lib_folder,
                  'nbs_path': nb_folder,
                  'doc_path': doc_folder,
                  'tst_flags': tst_flags}
        save_config_file(path/'config.yml', config)
        return cls()

    def save(self): save_config_file(self.config_file,self.d)

In [37]:
cfg = Config.create("nbdev", "https://github.com/fastai/nbdev/tree/master/", path='..', tst_flags=['tst'])
test_eq(Config().lib_name, 'nbdev')
test_eq(Config().git_url, "https://github.com/fastai/nbdev/tree/master/")
test_eq(Config().lib_path, Path.cwd().parent/'nbdev')
test_eq(Config().nbs_path, Path.cwd())
test_eq(Config().doc_path, Path.cwd().parent/'docs')

## Notebook checks

In [22]:
# export
def in_ipython():
    "Check if the code is running in the ipython environment (jupyter including)"
    program_name = os.path.basename(os.getenv('_', ''))
    if ('jupyter-notebook' in program_name or # jupyter-notebook
        'ipython'          in program_name or # ipython
        'JPY_PARENT_PID'   in os.environ):    # ipython-notebook
        return True
    else:
        return False

IN_IPYTHON = in_ipython()

In [23]:
# export
def in_colab():
    "Check if the code is running in Google Colaboratory"
    try:
        from google import colab
        return True
    except: return False

IN_COLAB = in_colab()

In [24]:
# export
def in_notebook():
    "Check if the code is running in a jupyter notebook"
    if in_colab(): return True
    try:
        shell = get_ipython().__class__.__name__
        if shell == 'ZMQInteractiveShell': return True   # Jupyter notebook, Spyder or qtconsole
        elif shell == 'TerminalInteractiveShell': return False  # Terminal running IPython
        else: return False  # Other type (?)
    except NameError: return False      # Probably standard Python interpreter

IN_NOTEBOOK = in_notebook()

In [25]:
assert IN_NOTEBOOK
assert not IN_COLAB
assert IN_IPYTHON

## Utils

In [30]:
# export
def last_index(x, o):
    "Finds the last index of occurence of `x` in `o` (returns -1 if no occurence)"
    res = len(o)-1
    while res >= 0 and o[res]!=x: res-=1
    return res

In [31]:
test_eq(last_index(1, [1,2,1,3,1,4]), 4)
test_eq(last_index(2, [1,2,1,3,1,4]), 1)
test_eq(last_index(5, [1,2,1,3,1,4]), -1)

The following is just for the purpose of testing things in the next notebooks

In [33]:
#export
def compose(*funcs, order=None):
    "Create a function that composes all functions in `funcs`, passing along remaining `*args` and `**kwargs` to all"
    if len(funcs)==0: return noop
    if len(funcs)==1: return funcs[0]
    def _inner(x, *args, **kwargs):
        for f in funcs: x = f(x, *args, **kwargs)
        return x
    return _inner

In [34]:
f1 = lambda o,p=0: (o*2)+p
f2 = lambda o,p=1: (o+1)/p
test_eq(f2(f1(3)), compose(f1,f2)(3))
test_eq(f2(f1(3,p=3),p=3), compose(f1,f2)(3,p=3))
test_eq(f2(f1(3,  3),  3), compose(f1,f2)(3,  3))

In [32]:
#export
class DocsTestClass:
    def test(): pass